In [1]:
from sympy import *
from IPython.display import display, Markdown, Math
from math import comb, prod
import itertools
import random
init_session(use_latex=True)

IPython console for SymPy 1.9 (Python 3.9.7-64-bit) (ground types: python)

These commands were executed:
>>> from __future__ import division
>>> from sympy import *
>>> x, y, z, t = symbols('x y z t')
>>> k, m, n = symbols('k m n', integer=True)
>>> f, g, h = symbols('f g h', cls=Function)
>>> init_printing()

Documentation can be found at https://docs.sympy.org/1.9/



In [2]:
def Vertices(*args):
    return 2 + sum([args[i] * (i+1) for i in range(len(args))])

def Faces(*args):
    return sum(args)

def Edges(*args):
    return Vertices(*args) + Faces(*args) - 1

def R(*args): # args =[m_2, m_3, ...]
    # (Edges-1)!/((Edges-(m_2+m_3+m_4))! m_2! m_3! ...)
    vertices = Vertices(*args) # (2+m_2+2m_3+3m_4+...+(n-1)m_n)
    faces = Faces(*args) # (m_2+m_3+m_4+...+m_n)
    edges = vertices + faces - 1 # (1+2m_2+3m_3+4m_4+...+n m_n)
    r = factorial(edges-1)//factorial(edges-faces) # binomial(edges, faces) * (vertices-1)!/edges
    for m in args:
        r //= factorial(m)
    return r


def Binom(*args): # args =[m2, m3, ...]
    vertices = Vertices(*args)
    faces = Faces(*args)
    edges = vertices + faces - 1
    n = edges - 1
    r = 1
    for m in args: 
        r *= comb(n, m)
        n -= m
    r //= vertices - 1
    return r


def powers(n, k, c, terms = []):
    if not terms:
        terms = [0 for i in range(c+1)]
    if (k == n):
        terms[1] = -(-1)**k
        return [tuple(terms)]
    if c == 1:
        return []
    valids = powers(n, k, c-1, terms[::])
    while (c <= n) and (k > 0):
        terms[c] += 1
        n -= c
        k -= 1
        valids += powers(n, k, c-1, terms[::])
    return valids + powers(n, k, c-1, terms[::])
        
def diagonal(n, c):
    q = 0
    p = set(powers(2*n, n, len(c)-1))
    for el in p:
        term = el[1] * Binom(*el[2:]) * c[1]**(n-sum(el[2:]))
        term *= prod([c[k]**el[k] for k in range(2,len(el))])
        q += term
    return q

def sol(n, c):
    return sum([c[0]**(i+1)/c[1]**(2*i+1)*diagonal(i,c) for i in range(n+1)])
        
def run():
    while True:
        steps = 0
        inp = input('Insert an expression with explicit math symbols (example: "x^5-10*x^3+.02*x^4+x**7"):\n')
        try:
            pol = Poly(sympify(inp), Symbol('x'))
            d = dict(pol.all_terms())
        except:
            print('Invalid expression. Turning off.')
            break
        values = [d[key] for key in sorted( d.keys())]
        if values[1] == 0:
            print('Invalid expression. Turn off.')
            break
        try:
            order = int(input('Insert the depth for the algorithm:\n'))
            assert(order > 0)
        except:
            print('Invalid input (must be a natural number).')
            break
        approx = Float(sol(order, values))
        tmp = Poly(pol.as_expr(),Symbol('x'))
        e = tmp.eval(approx)
        while abs(e) > 0.000001:
            steps += 1
            display(
                Markdown(
                        "Couldn't find a good approximation yet for p(x) = "+\
                        '{}'.format(latex(sympify(pol.as_expr()), mode='inline'))
                ),
                Markdown(
                'p({}) = '.format(latex(approx, mode='inline'))+\
                '{}'.format(latex(Float(e,30), mode='inline'))
                ),
                Markdown(
                        "Let's use the previous result as an approximation (step {})".format(latex(steps, mode='inline')))
            )
            tmp = Poly(pol.as_expr(),Symbol('x'))
            d = dict(Poly(tmp.replace(Symbol('x'), Symbol('x')+approx), Symbol('x')).all_terms())
            values = [d[key] for key in sorted( d.keys())]
            approx = approx+sol(order, values)
            e = tmp.eval(approx)
        display(
            Markdown(
                    "Found a good approximation for p(x) = "+\
                    '{}'.format(latex(sympify(pol.as_expr()), mode='inline')))
            )
        display(N(approx))
        display(
            Markdown(
                'p({}) = '.format(latex(approx, mode='inline'))+\
                '{}'.format(latex(Float(e,30), mode='inline'))))
        print('\n\n\n\n')
            
# 3-4*x + x^2-2*x^3+x^4+3*x^5
#10.1+3.7*x+3*x**2-4*x^3-20.5*x**4+x**5+6.1*x**6-2.2*x**7-x**8+2.5*x^9-x**10

In [4]:
run()

Insert an expression with explicit math symbols (example: "x^5-10*x^3+.02*x^4+x**7"):
3-4*x + x^2-2*x^3+x^4+3*x^5
Insert the depth for the algorithm:
4


Couldn't find a good approximation yet for p(x) = $3 x^{5} + x^{4} - 2 x^{3} + x^{2} - 4 x + 3$

p($0.940681457519531$) = $1.45009214915946516022415835323$

Let's use the previous result as an approximation (step $1$)

Couldn't find a good approximation yet for p(x) = $3 x^{5} + x^{4} - 2 x^{3} + x^{2} - 4 x + 3$

...

p($20.6581165174639$) = $4889.18894538232052582316100597$

Let's use the previous result as an approximation (step $95$)

Found a good approximation for p(x) = $1.0 x^{5} - 20.5 x^{4} - 4.0 x^{3} + 3.0 x^{2} + 3.7 x + 10000.0$

p($20.631216413435$) = $0.000000003390596248209476470947265625$






Insert an expression with explicit math symbols (example: "x^5-10*x^3+.02*x^4+x**7"):
3d
Invalid expression. Turning off.
